#Escher Tools
Tools for creating Escher transformations.
##Version History
###V6
- Changed to vertical words.

###V5
- Added `spacepine`, `locatepine`, and `looppine` to support transforming to and from spaces in words.

### V4
- Updated `strtrans` to use one letter (specified in arguments) to determine letter height and width.

###V3
- Removed `chartrans` and placed functionality in `strtrans`.
- Updated `strtrans` to only 1 file input.
- Updated `strtrans` with boolean `topines` option.
- Add inverted pinetree support to `compile`.
- Changed default `vscale` in `compile` to 1.2.

###V2
- Added pine trees to `chartrans`.
- Moved `findmidpoints` from `Escher Tools` to `Img Tools`.
- Moved `resdown` and `ressame` from `ResDown` to `Escher Tools`.

###V1
- Updated `spaces` and `letters` to save characters as "filename_n"and "filename-n" respectively.
- Updated `findmidpoints` to include the spaces at begining and end of word.
- Updated `strtrans` and `chartrans` to act on files, not strings.

###V0
- Initial release
- `chartrans`, `strtrans`, and `compile` copied from `Escher Transformations V2`.

In [11]:
function spacepine(xcenter, ycenter, height, width, res, topine::Bool, n=10)
    #returns transform to/from pine at center with given 'width' and 'height' of resoultion
    
    #space
    spacex = fill(xcenter, res)
    spacey = fill(ycenter, res)
    
    #tree
    pinex, piney = pinetree(height, width, res, !topine) #plant
    pinex .+= (xcenter - width/2) #transplant x
    
    if topine == false #flip pine over
        piney *= -1
        piney .+= (ycenter + height/2)
    else
        piney .+= (ycenter - height/2) #transplant y
    end
        
    #transform
    x = vlinspace(pinex, spacex, n)
    y = vlinspace(piney, spacey, n)
    
    if topine == true #flip order if necessary
        x = fliplr(x)
        y = fliplr(y)
    end
    
    return x, y
end

spacepine (generic function with 2 methods)

In [2]:
function locatepine(file::String, spaceloc, path="\\")
    #Returns dimensions and center between the letter at 'spaceloc' and the next letter
    
    #load images
    loc = pwd() * path * file[1:end-4] * "-" * string(spaceloc) * ".bmp" #location of file
    edge1 = edgedetect(imread(loc))
    
    loc = pwd() * path * file[1:end-4] * "-" * string(spaceloc + 1) * ".bmp" #location of file 
    edge2 = edgedetect(imread(loc))
    
    #find boundaries
    left = maximum(edge1[1])
    right = minimum(edge2[1])
    top = maximum(edge1[2])
    bottom = minimum(edge1[2])
    
    #calculate
    width = (left - minimum(edge1[1])) * 4 / 3
    height = (top - bottom) * 1.2
    xcenter = (left + right) / 2
    ycenter = (top + bottom) / 2 - 0.1 * (top - bottom)
       
    return xcenter, ycenter, height, width
end

locatepine (generic function with 2 methods)

In [3]:
function looppine(file::String, spaces, res, topine, path="\\", n=10)
    #returns x and y coordinates of transformations to/from ('topine') from/to a space located after 'spaces'.
    
    x = zeros(Float64, res * length(spaces),n)
    y = zeros(Float64, res * length(spaces),n)
    
    for i = 1:length(spaces) #loop through each space
        xcenter, ycenter, height, width = locatepine(file, spaces[i], path)
        x[(i-1)*res+1:i*res,:], y[(i-1)*res+1:i*res,:] = spacepine(xcenter, ycenter, height, width, res, topine, n)
    end
    return x, y
end

looppine (generic function with 3 methods)

In [17]:
function strtrans(file::String, topine::Bool, templateletter = 1, path="\\", n=10)
    #morphs 'file1' (located at 'path') to or from (according to 'topines') in 'n' steps 
    #'topine' specifies direction of tree. 'true' give an upside down tree
    #'templateletter' specifies which letter to look at to determine master letter size
    
    i=1 #loop counter
    transx = Array(Float64, 0, n) #x-locations of data points
    transy = Array(Float64, 0, n)
    #img = imread(pwd() * path * file) #initialize image
    
    #find letter size
    loc = pwd() * path * file[1:end-4] * "-" * string(templateletter) * ".bmp" #location of file
    img = imread(loc)
    edge = edgedetect(img)
    edge = hcat(edge[1],edge[2])
    letterwidth = (maximum(edge[:,1]) - minimum(edge[:,1]))
    letterheight = (maximum(edge[:,2]) - minimum(edge[:,2]))

    #loop through letters
    while true #loop until no more files exist
        
        loc = pwd() * path * file[1:end-4] * "-" * string(i) * ".bmp" #location of file

        #load image
        try #loop until files are gone
            img = imread(loc)
        catch e
            return transx, transy
        end
        
        edge = edgedetect(img)
        edge = hcat(edge[1],edge[2])
        
        #plant pine tree
        pinex, piney = pinetree(1.2 * letterheight, 5/4 * letterwidth, size(edge)[1], !topine) #grow a tree
        #piney .+=  (minimum(edge[:,2]) - 0.1 * letterheight) #x-transplant tree in center of letter
        
        #if topine == true #transforming to pine
            #piney .+= 0.1 * letterheight #y-transplant to center
        #else #transforming from pine
        piney *= -1 #flip tree
        piney .+= (1.2 * i * letterheight + 0.077/2 * letterheight)
        #piney .+= 1.3 * letterheight * (i-1)#y-transplant to center
            
        #end

        #transform and concatenate
        if topine == true #transforming to pine
            x = vlinspace(edge[:,1], pinex, n)
            y = vlinspace(edge[:,2], piney, n)
        else #transforming from pine
            x = vlinspace(pinex, edge[:,1], n)
            y = vlinspace(piney, edge[:,2], n)
        end

        transx = vcat(transx, x)
        transy = vcat(transy, y)   
        
        i += 1
        
        if i > 1e3
            error("stuck in loop")
        end
        
        
    end      
end

strtrans (generic function with 4 methods)

In [14]:
function compile(x, y, topine=true, vscale=1.25, step = -1.)
    #stacks morphed frames on top of each other (with vertical scale 'vscale')
    
    allx = zeros(Float64, size(x)[1]*size(x)[2])
    ally = zeros(Float64, size(x)[1]*size(x)[2])
    points = size(x)[1]
    if step < 0
    #if topine == true #top half of transform: look at top
    step = (maximum(x)-minimum(x))
    #else #bottom half of transform: look at bottom
    #    step = (maximum(y[:,end])-minimum(y[:,end]))*vscale
    #end
    end
    for n = 1:size(x)[2]
        allx[(n-1)*points+1:n*points] = x[:,n]+step*(n-1)
        ally[(n-1)*points+1:n*points] = y[:,n]
    end
    
    return allx, ally
end

compile (generic function with 4 methods)

In [8]:
function letters(file, spacemidpoints, path="\\")
    #saves individual files (at 'path') for each letter in 'file' with everything else blacked out
    
    img = imread(pwd()*path*file) #open image

    for i = 2:length(spacemidpoints)
        letter = zeros(img) #intialize all black

        letter[:,spacemidpoints[i-1]:spacemidpoints[i]] = invert(img[:,spacemidpoints[i-1]:spacemidpoints[i]]) #fill in white letter
        letter[1,:] = zeros(letter[1,:]) #black left column
        letter[end,:] = zeros(letter[end,:]) #black right column
        imwrite(letter', pwd()*path*file[1:end-4]*"-"*string(i-1)*".bmp") #save file with letter name
    end
end

letters (generic function with 2 methods)

In [9]:
function spaces(file, lettermidpoints, path="\\")
    #saves individula files (at 'path') for each space in 'file' with everything else blacked out
    
    img = imread(pwd()*path*file) #open image

    for i = 1:length(lettermidpoints)-1
        space = zeros(img) #intialize all black

        space[lettermidpoints[i]:lettermidpoints[i+1],:] = img[lettermidpoints[i]:lettermidpoints[i+1],:] #fill in 
        imwrite(space', pwd()*path*file[1:end-4]*"_"*string(i)*".bmp") #save file with letter name
    end
end


spaces (generic function with 2 methods)

In [10]:
function resdown(x, newsize)
    #deletes evenly spaced points from x to reduce it to 'newsize' in dimension 1
    oldsize = size(x)[1]
    Δ = oldsize - newsize #change in size
    step = oldsize/Δ #step size between deleted points. will fail when this is less than two?
    y = zeros(newsize, 2) #output vector
    
    for n=0:Δ-1 #pull segment of x to place in y
        y[ifloor(step*n+1-n):ifloor(step*(n+1)-n-1),:] = x[ifloor(step*n+1):ifloor(step*(n+1)-1),:]
    end
    
    return y
end

resdown (generic function with 1 method)

In [11]:
function ressame(x,y)
    #deletes evenly spaced points from the shorter of 'x' and 'y' to make them the same length
    xsize = size(x)[1]
    ysize = size(y)[1]
    if xsize > ysize
        x = resdown(x, ysize)
    elseif ysize > xsize
        y = resdown(y, xsize)
    end
    return x, y
end


ressame (generic function with 1 method)